In [1]:
import torch

C:\Users\emile\Miniconda3\envs\torch\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [5]:
from torch import nn, optim

In [6]:
from torch.utils import data

In [7]:
from loguru import logger

In [8]:
import visualize

In [9]:
import numpy as np

In [10]:
import time

In [11]:
import torchvision
import torchvision.transforms

In [12]:
import resnet, base

In [33]:
def learning_rate_schedules(batch_num, initial_lr, warmup_batchs, total_batchs, decrease_schedules):
    """
    """
    if batch_num < warmup_batchs:
        lr = initial_lr * (batch_num+1)/warmup_batchs
    else:
        num = batch_num - warmup_batchs + 1
        lr = 0.5 * (1 + math.cos(math.pi * (num / total_batchs))) * initial_lr
    return lr

In [13]:
root = 'C:/Datasets/CIFAR10'

In [14]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [15]:
train_transform = torchvision.transforms.Compose(
    
    [torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(size=[32,32], padding=4),
    torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))
    ])

In [16]:
test_transform= torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))])

In [17]:
train_dataset = torchvision.datasets.CIFAR10(root, transform=train_transform, download=True)

Files already downloaded and verified


In [18]:
img, label = train_dataset[0]

In [19]:
test_dataset = torchvision.datasets.CIFAR10(root, train=False, transform=test_transform, download=True)

Files already downloaded and verified


In [20]:
train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))

In [21]:
net = resnet.ResNet32()

2019-06-25 17:48:18.110 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-25 17:48:18.111 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-25 17:48:18.113 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-25 17:48:18.114 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-25 17:48:18.116 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-25 17:48:18.117 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 32, stride: 2
2019-06-25 17:48:18.119 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channels: 32, stride: 1
2019-06-25 17:48:18.121 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channels: 32, stride: 1
2019-06-25 17:48:18.123 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channel

In [25]:
learning_rate = 0.001

In [34]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [35]:
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [36]:
batch_size = 512

In [40]:
epoch = 50

In [41]:
use_cuda = net.use_cuda

In [42]:
if use_cuda:
    net = net.cuda()

In [43]:
use_cuda = net.use_cuda

In [44]:
visualizer = visualize.Visualizer('Net')

In [ ]:
for e in range(epoch):
    start_time = time.time()
    net.train()
    train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))
    train_data_iter = data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    running_loss = 0.0
    for i, (train_data, label) in enumerate(train_data_iter):
        if use_cuda:
            train_data = train_data.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        prediction = net(train_data)
        loss = net.loss(prediction, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if e > 1 and e % 20 == 0:
        learning_rate = learning_rate / 5
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    valid_train_res, valid_ap = base.validate(net, train_dataset, batch_size, num_class=10)
    valid_test_res, test_ap = base.validate(net, test_dataset, batch_size, num_class=10)
    loss_dct = {'loss': running_loss}
    val_precision = valid_train_res['map']
    val_ap_dct = {classes[i]: v for i, v in enumerate(valid_ap)}
    test_precision = valid_test_res['map']
    logger.info(f'epoch num: {e}, loss: {running_loss}, val_precision: {val_precision}, test_precision: {test_precision}, learning_rate: {learning_rate}, time: {time.time() - start_time}')
    visualizer.plot(loss_dct)
    visualizer.plot({'val_precision': val_precision})
    visualizer.plot(val_ap_dct)
    visualizer.plot({'test_precision': test_precision})

2019-06-25 17:51:09.500 | INFO     | __main__:<module>:27 - epoch num: 0, loss: 151.94620215892792, val_precision: 0.2940800189971924, test_precision: 0.2955000102519989, learning_rate: 0.001, time: 71.08110094070435
2019-06-25 17:52:23.224 | INFO     | __main__:<module>:27 - epoch num: 1, loss: 141.88241505622864, val_precision: 0.3271600306034088, test_precision: 0.3274000287055969, learning_rate: 0.001, time: 73.6539318561554


In [149]:
torch.save(net.state_dict(), 'audio_net_002.save')

In [2]:
{'loss': 529.4704940319061, 'val_precision': 0.78054, 'test_precision': 0.6408, 'learning_rate': 0.01, 'time': 19.557708978652954}

{'loss': 529.4704940319061,
 'val_precision': 0.78054,
 'test_precision': 0.6408,
 'learning_rate': 0.01,
 'time': 19.557708978652954}

In [232]:
resnet34 = torchvision.models.resnet34(pretrained=False)

In [234]:
resnet50 = torchvision.models.resnet50(pretrained=False)

In [235]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [250]:
sum(p.numel() for p in resnet50.parameters() if p.requires_grad)

25557032

In [249]:
num

25557032